In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import warnings
warnings.filterwarnings('ignore')
import emojis
import unicodedata
from bs4 import BeautifulSoup
from langdetect import detect, DetectorFactory
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
nltk.download('wordnet')
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
# Libraries for deep learning
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, Embedding, Dropout, Bidirectional, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

/Users/tienhuynh/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tienhuynh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2024-09-27 17:00:46.113838: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
review = pd.read_csv("./review_cleaned.csv")
review

,id,name,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,award,...,value,location,sleep_quality,rooms,check_in_front_desk,business_service,ratings,normalized_review,normalized_description,normalized_name
0,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,We had a whisper-quiet queen deluxe room couty...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,5.0,5.0,4.0,4.0,5.0,whisper quiet queen deluxe room coutyard room ...,experience magic time square step door escape ...,casablanca hotel time square
1,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,Great 3 night stay right after Thanksgiving. A...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,5.0,4.0,4.0,4.0,5.0,great night stay right thanksgiving agree prev...,experience magic time square step door escape ...,casablanca hotel time square
2,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,I hesitate to give this little gem a stellar r...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,5.0,5.0,4.0,4.0,5.0,hesitate give little gem stellar review fear a...,experience magic time square step door escape ...,casablanca hotel time square
3,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,You won't find a more conveniently located hot...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,5.0,4.0,4.0,4.0,5.0,find conveniently locate hotel nyc pace count ...,experience magic time square step door escape ...,casablanca hotel time square
4,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,This small boutique hotel right off Times Squa...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,4.0,4.0,4.0,4.0,5.0,small boutique hotel right time square perfect...,experience magic time square step door escape ...,casablanca hotel time square
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239161,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,Beautifully decorated and comfortable rooms ev...,4.0,79,very expensive,347.214,3,1,...,3.9,5.0,5.0,4.0,4.0,4.0,5.0,beautifully decorate comfortable room even tho...,sofitel washington dc lafayette square represe...,sofitel washington dc
239162,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,I was there with my girlfriend for 4 days at t...,4.0,79,very expensive,347.214,3,1,...,3.9,5.0,5.0,4.0,4.0,4.0,5.0,girlfriend day end march location great close ...,sofitel washington dc lafayette square represe...,sofitel washington dc
239163,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,Upscale hotel in downtown DC. Walking distance...,4.0,79,very expensive,347.214,3,1,...,3.9,5.0,5.0,4.0,4.0,4.0,4.0,upscale hotel downtown dc walk distance white ...,sofitel washington dc lafayette square represe...,sofitel washington dc
239164,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,All expectations for service and accommodation...,4.0,79,very expensive,347.214,3,1,...,3.9,5.0,5.0,4.0,4.0,4.0,5.0,expectation service accommodation exceed probl...,sofitel washington dc lafayette square represe...,sofitel washington dc


In [3]:
review['ratings'].value_counts()

ratings
5.0    101589
4.0     76777
3.0     31063
2.0     15680
1.0     14057
Name: count, dtype: int64

In [4]:
def categorize_rating(rating):
    if rating <= 3:
        return 0
    else:
        return 1

# Apply the categorization function to the ratings column
review['target'] = review['ratings'].apply(categorize_rating)

In [5]:
review['target'].value_counts(normalize=True)

target
1    0.745783
0    0.254217
Name: proportion, dtype: float64

In [6]:
X = review['normalized_review']
y = review['target']

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
y_train.value_counts(normalize=True)

target
1    0.745753
0    0.254247
Name: proportion, dtype: float64

In [9]:
# Combine X_train and y_train to apply resampling
train = pd.concat([X_train, y_train], axis=1)

# Separate majority and minority classes
majority_class = train[train['target'] == 1]  # Assuming 0 is the majority class
minority_class = train[train['target'] == 0]  # Assuming 1 is the minority class

# Apply upsampling or downsampling
# Example: Upsampling
minority_upsampled = resample(minority_class, 
                              replace=True,     # sample with replacement
                              n_samples=len(majority_class),  # match to majority class
                              random_state=42)

# Combine majority class with upsampled minority class
train_upsampled = pd.concat([majority_class, minority_upsampled])

# Separate X_train and y_train again
X_train_balanced = train_upsampled.drop('target', axis=1)
y_train_balanced = train_upsampled['target']

In [43]:
y_train_balanced.value_counts(normalize=True)

target
1    0.5
0    0.5
Name: proportion, dtype: float64

In [11]:
#Countvectorizer
# Use count vectorizer to check how many unique words there are
cvec = CountVectorizer(stop_words='english') 
cvec_df = pd.DataFrame(cvec.fit_transform(X_train_balanced['normalized_review']).todense(), columns=cvec.get_feature_names_out())
cvec_df.shape

(249702, 79436)

In [12]:
def cmat(actual_y, predictions, dataset):
    # Check the unique classes in the actual values and predictions
    unique_classes = np.unique(np.concatenate((actual_y, predictions)))
    num_classes = len(unique_classes)
    
    # Create a classification report
    print('Classification report for', dataset)
    print(classification_report(actual_y, predictions))
    print('')
    
    # Create a confusion matrix
    cm = confusion_matrix(actual_y, predictions)
    
    # Dynamically create labels for the confusion matrix
    labels = [f'Class {cls}' for cls in unique_classes]
    
    # Convert confusion matrix to DataFrame
    cm_df = pd.DataFrame(cm, columns=[f'Predicted {label}' for label in labels], 
                         index=[f'Actual {label}' for label in labels])
    
    print('Confusion matrix for', dataset)
    print(cm_df)


In [13]:
# Create a pipeline with Count Vectorizer and Logistic Regression
pipe_cvec_lr = Pipeline([
    ('cvec', CountVectorizer(stop_words='english')), 
    ('lr', LogisticRegression(random_state=42))
])

# Search over the following values of hyperparameters:
pipe_cvec_lr_params = {
    'cvec__max_features': [300], #100,200
    'cvec__min_df': [0.001, 0.005], 
    'cvec__max_df': [.9, .95], 
    'cvec__ngram_range':[(1,1),(1,2)],  
    'lr__penalty': ['l2'],
    'lr__C': [.01,.1]
}

# Instantiate GridSearchCV
gs_cvec_lr = GridSearchCV(pipe_cvec_lr, # Objects to optimise
                          param_grid = pipe_cvec_lr_params, # Hyperparameters for tuning
                          cv=10) # 10-fold cross validation

# Fit model on to training data
gs_cvec_lr.fit(X_train_balanced.values.ravel(), y_train_balanced)

# Generate predictions on validation set
cvec_lr_pred = gs_cvec_lr.predict(X_test)

# Print best parameters
print('Best parameters: ', gs_cvec_lr.best_params_)

# Print accuracy scores
print('Best CV score: ', gs_cvec_lr.best_score_)
print('Training score:', gs_cvec_lr.score(X_train_balanced.values.ravel(), y_train_balanced))
print('Validation score:', gs_cvec_lr.score(X_test, y_test))
print('')

# Print classification report and confusion matrix
cmat(y_test, cvec_lr_pred, 'validation set')

Best parameters:  {'cvec__max_df': 0.9, 'cvec__max_features': 300, 'cvec__min_df': 0.001, 'cvec__ngram_range': (1, 1), 'lr__C': 0.1, 'lr__penalty': 'l2'}
Best CV score:  0.8309785387092278
Training score: 0.83159926632546
Validation score: 0.8350662020905923

Classification report for validation set
              precision    recall  f1-score   support

           0       0.63      0.83      0.72     18235
           1       0.93      0.84      0.88     53515

    accuracy                           0.84     71750
   macro avg       0.78      0.83      0.80     71750
weighted avg       0.86      0.84      0.84     71750


Confusion matrix for validation set
                Predicted Class 0  Predicted Class 1
Actual Class 0              15080               3155
Actual Class 1               8679              44836


In [14]:
# Create a pipeline with Count Vectorizer and Naive Bayes
pipe_cvec_nb = Pipeline([
    ('cvec', CountVectorizer(stop_words='english')),
    ('nb', MultinomialNB())
])

# Search over the following values of hyperparameters:
pipe_cvec_nb_params = {
    'cvec__max_features': [500], #200
    'cvec__min_df': [0.001, 0.005],
    'cvec__max_df': [.9,.95], 
    'cvec__ngram_range':[(1,1),(1,2)],  
}

# Instantiate GridSearchCV
gs_cvec_nb = GridSearchCV(pipe_cvec_nb, # Objects to optimise
                          param_grid = pipe_cvec_nb_params, # Hyperparameters for tuning
                          cv=10) # 10-fold cross validation

# Fit model on to training data
gs_cvec_nb.fit(X_train_balanced.values.ravel(), y_train_balanced)

# Generate predictions on validation set
cvec_nb_pred = gs_cvec_nb.predict(X_test)

# Print best parameters
print('Best parameters: ', gs_cvec_nb.best_params_)

# Print accuracy scores
print('Best CV score: ', gs_cvec_nb.best_score_)
print('Training score:', gs_cvec_nb.score(X_train_balanced.values.ravel(), y_train_balanced))
print('Validation score:', gs_cvec_nb.score(X_test, y_test))
print('')

# Print classification report and confusion matrix
cmat(y_test, cvec_nb_pred, 'validation set')

Best parameters:  {'cvec__max_df': 0.9, 'cvec__max_features': 500, 'cvec__min_df': 0.005, 'cvec__ngram_range': (1, 1)}
Best CV score:  0.8215032343405415
Training score: 0.8213910981890413
Validation score: 0.8479721254355401

Classification report for validation set
              precision    recall  f1-score   support

           0       0.67      0.78      0.72     18235
           1       0.92      0.87      0.90     53515

    accuracy                           0.85     71750
   macro avg       0.80      0.82      0.81     71750
weighted avg       0.86      0.85      0.85     71750


Confusion matrix for validation set
                Predicted Class 0  Predicted Class 1
Actual Class 0              14142               4093
Actual Class 1               6815              46700


In [15]:
# Create a pipeline with TF-IDF and Logistic Regression
pipe_tvec_lr = Pipeline([
    ('tvec', TfidfVectorizer(stop_words='english')),
    ('lr', LogisticRegression(random_state=42))
])

# Search over the following values of hyperparameters:
pipe_tvec_lr_params = {
    'tvec__max_features': [300], #100,200
    'tvec__min_df': [0.001, 0.005], #2,3 
    'tvec__max_df': [.9,.95], 
    'tvec__ngram_range':[(1,1),(1,2)],  
    'lr__penalty': ['l2'],
    'lr__C': [.1, 1] #.1, .01
}

# Instantiate GridSearchCV
gs_tvec_lr = GridSearchCV(pipe_tvec_lr, # Objects to optimise
                          param_grid = pipe_tvec_lr_params, # Hyperparameters for tuning
                          cv=10) # 10-fold cross validation

# Fit model on to training data
gs_tvec_lr.fit(X_train_balanced.values.ravel(), y_train_balanced)

# Generate predictions on validation set
tvec_lr_pred = gs_tvec_lr.predict(X_test)

# Print best parameters
print('Best parameters: ', gs_tvec_lr.best_params_)

# Print accuracy scores
print('Best CV score: ', gs_tvec_lr.best_score_)
print('Training score:', gs_tvec_lr.score(X_train_balanced.values.ravel(), y_train_balanced))
print('Validation score:', gs_tvec_lr.score(X_test, y_test))
print('')

# Print classification report and confusion matrix
cmat(y_test, tvec_lr_pred, 'validation set')

Best parameters:  {'lr__C': 1, 'lr__penalty': 'l2', 'tvec__max_df': 0.9, 'tvec__max_features': 300, 'tvec__min_df': 0.001, 'tvec__ngram_range': (1, 1)}
Best CV score:  0.8319356688440209
Training score: 0.8325363833689758
Validation score: 0.8332682926829268

Classification report for validation set
              precision    recall  f1-score   support

           0       0.63      0.84      0.72     18235
           1       0.94      0.83      0.88     53515

    accuracy                           0.83     71750
   macro avg       0.78      0.83      0.80     71750
weighted avg       0.86      0.83      0.84     71750


Confusion matrix for validation set
                Predicted Class 0  Predicted Class 1
Actual Class 0              15245               2990
Actual Class 1               8973              44542


In [16]:
# Create a pipeline with TF-IDF and Naive Bayes
pipe_tvec_nb = Pipeline([
    ('tvec', TfidfVectorizer(stop_words='english')),
    ('nb', MultinomialNB())
])

# Search over the following values of hyperparameters:
pipe_tvec_nb_params = {
    'tvec__max_features': [500], #200
    'tvec__min_df': [0.001, 0.005], #
    'tvec__max_df': [.9,.95], 
    'tvec__ngram_range':[(1,1),(1,2)],  
}

# Instantiate GridSearchCV
gs_tvec_nb = GridSearchCV(pipe_tvec_nb, # Objects to optimise
                          param_grid = pipe_tvec_nb_params, # Hyperparameters for tuning
                          cv=10) # 10-fold cross validation

# Fit model on to training data
gs_tvec_nb.fit(X_train_balanced.values.ravel(), y_train_balanced)

# Generate predictions on validation set
tvec_nb_pred = gs_tvec_nb.predict(X_test)

# Print best parameters
print('Best parameters: ', gs_tvec_nb.best_params_)

# Print accuracy scores
print('Best CV score: ', gs_tvec_nb.best_score_)
print('Training score:', gs_tvec_nb.score(X_train_balanced.values.ravel(), y_train_balanced))
print('Validation score:', gs_tvec_nb.score(X_test, y_test))
print('')

# Print classification report and confusion matrix
cmat(y_test, tvec_nb_pred, 'validation set')

Best parameters:  {'tvec__max_df': 0.9, 'tvec__max_features': 500, 'tvec__min_df': 0.001, 'tvec__ngram_range': (1, 1)}
Best CV score:  0.8270378306837533
Training score: 0.8270498434133488
Validation score: 0.8372682926829268

Classification report for validation set
              precision    recall  f1-score   support

           0       0.64      0.81      0.72     18235
           1       0.93      0.85      0.89     53515

    accuracy                           0.84     71750
   macro avg       0.79      0.83      0.80     71750
weighted avg       0.86      0.84      0.84     71750


Confusion matrix for validation set
                Predicted Class 0  Predicted Class 1
Actual Class 0              14828               3407
Actual Class 1               8269              45246


In [17]:
# Instantiate the sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

In [18]:
# Create an empty list to append the polarity scores
sia_list = []

# Loop through the text in the training dataset and create a dictionary of the VADER scores
for text in review['normalized_review']:
    sia_dict = {}
    sia_dict['neg'] = sia.polarity_scores(text)['neg'] # Negative score
    sia_dict['neu'] = sia.polarity_scores(text)['neu'] # Neutral score
    sia_dict['pos'] = sia.polarity_scores(text)['pos'] # Positive scores
    sia_dict['compound'] = sia.polarity_scores(text)['compound'] # Compound scores
    sia_list.append(sia_dict) # Append the dictionary of scores to the sia_list

In [19]:
review['normalized_review'].isnull().sum()

0

In [20]:
# Create a dataframe from the sia_list
sia_df = pd.DataFrame(sia_list)

In [22]:
sia_df['content'] = review['normalized_review']
sia_df['target'] = review['target'] 


In [65]:
sia_df

,neg,neu,pos,compound,content,target
0,0.000,0.674,0.326,0.9468,whisper quiet queen deluxe room coutyard room ...,1
1,0.017,0.688,0.295,0.9705,great night stay right thanksgiving agree prev...,1
2,0.151,0.627,0.222,0.3612,hesitate give little gem stellar review fear a...,1
3,0.117,0.604,0.279,0.8957,find conveniently locate hotel nyc pace count ...,1
4,0.000,0.709,0.291,0.9826,small boutique hotel right time square perfect...,1
...,...,...,...,...,...,...
239161,0.000,0.554,0.446,0.9750,beautifully decorate comfortable room even tho...,1
239162,0.051,0.553,0.395,0.9413,girlfriend day end march location great close ...,1
239163,0.000,0.617,0.383,0.9468,upscale hotel downtown dc walk distance white ...,1
239164,0.089,0.757,0.154,0.2472,expectation service accommodation exceed probl...,1


In [79]:
def extract_score(compound):
    sentiment = ''
    
    if compound >= 0.05:
        sentiment = "positive"
    elif compound <= -0.05:
        sentiment = "negative"

    return sentiment


In [80]:
review['sentiment'] = sia_df['compound'].apply(extract_score)

In [91]:
review

,id,name,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,award,...,sleep_quality,rooms,check_in_front_desk,business_service,ratings,normalized_review,normalized_description,normalized_name,target,sentiment
0,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,We had a whisper-quiet queen deluxe room couty...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,4.0,4.0,5.0,whisper quiet queen deluxe room coutyard room ...,experience magic time square step door escape ...,casablanca hotel time square,1,positive
1,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,Great 3 night stay right after Thanksgiving. A...,4.0,50,very expensive,130.960,5,1,...,5.0,4.0,4.0,4.0,5.0,great night stay right thanksgiving agree prev...,experience magic time square step door escape ...,casablanca hotel time square,1,positive
2,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,I hesitate to give this little gem a stellar r...,4.0,50,very expensive,130.960,5,1,...,5.0,5.0,4.0,4.0,5.0,hesitate give little gem stellar review fear a...,experience magic time square step door escape ...,casablanca hotel time square,1,positive
3,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,You won't find a more conveniently located hot...,4.0,50,very expensive,130.960,5,1,...,5.0,4.0,4.0,4.0,5.0,find conveniently locate hotel nyc pace count ...,experience magic time square step door escape ...,casablanca hotel time square,1,positive
4,113317,Casablanca Hotel Times Square,Experience the magic of Times Square just step...,This small boutique hotel right off Times Squa...,4.0,50,very expensive,130.960,5,1,...,4.0,4.0,4.0,4.0,5.0,small boutique hotel right time square perfect...,experience magic time square step door escape ...,casablanca hotel time square,1,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239161,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,Beautifully decorated and comfortable rooms ev...,4.0,79,very expensive,347.214,3,1,...,5.0,4.0,4.0,4.0,5.0,beautifully decorate comfortable room even tho...,sofitel washington dc lafayette square represe...,sofitel washington dc,1,positive
239162,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,I was there with my girlfriend for 4 days at t...,4.0,79,very expensive,347.214,3,1,...,5.0,4.0,4.0,4.0,5.0,girlfriend day end march location great close ...,sofitel washington dc lafayette square represe...,sofitel washington dc,1,positive
239163,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,Upscale hotel in downtown DC. Walking distance...,4.0,79,very expensive,347.214,3,1,...,5.0,4.0,4.0,4.0,4.0,upscale hotel downtown dc walk distance white ...,sofitel washington dc lafayette square represe...,sofitel washington dc,1,positive
239164,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square represe...,All expectations for service and accommodation...,4.0,79,very expensive,347.214,3,1,...,5.0,4.0,4.0,4.0,5.0,expectation service accommodation exceed probl...,sofitel washington dc lafayette square represe...,sofitel washington dc,1,positive


In [92]:
review.to_csv("review_cleaned.csv", index=False)